# Api Nifi

## Librerias

In [22]:
from urllib import response
import requests
import json
import urllib3
from IPython.display import display
import time
import sys
import warnings
##desactivamos los mensajes de peligro
from prettytable import PrettyTable
from prettytable.colortable import RESET_CODE, ColorTable, Theme
urllib3.disable_warnings()

## Variables Globales

In [23]:
USERNAME = "EXD21089"
PASSWORD = "Vegito7284"
URL = "https://cdf01it-01adl.claro.amx:8443/nifi-api"

## Recorrido

### Acceso al token de la api

In [24]:
#Obtención del token de nifi
def obtenerToken(URL):
    ingreso = "/access/token?username={}&password={}".format(USERNAME, PASSWORD)
    HeadersSinJson = {'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8', 'User-Agent': 'PostmanRuntime/7.29.0'}
    response = requests.post(URL+ingreso, headers=HeadersSinJson, verify = False)        
    if response.status_code == 201 or response.status_code == 200:
       #devuelve el token en byte, no como string
        base = response.content
        token = base.decode("utf-8")
        return token
    else:
        print(response.status_code)
    
    #Recorremos el archivo con el escenario de prueba

TokenNifi = obtenerToken(URL)

## Diccionarios y tablas

### Diccionarios y lector de los procesadores

In [25]:
""" 
    @uthor: Leonardo Alverto Rébola
    Description: Creacion de una clase de diccionario
"""
class CustomDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value

        
class TableDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value
        
class GroupsDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value
        
#instancia del dict
dict_proccessors = CustomDict()
    
#desglozamos y buscamos los datos de los proccessors
def loadProccessors(pg):
    if type(pg) == int:
        return False
    else:
        proccessors = pg['processGroupFlow']['flow']['processors']
        for proccessor in proccessors:
            ##error add() takes 3 positional arguments but 4 were given puede ser por que dentro revision tiene dos valores
            dict_proccessors.add(proccessor['component']['name'], proccessor['component']['id'])                    
        return dict_proccessors

### Diccionario y lector de conectores

In [26]:
class Connection:
    def __init__(self,id, name, sourceId, sourceName, destinationId, destinationName):
        self.id = id
        self.name = name
        self.sourceId = sourceId
        self.sourceName = sourceName
        self.destinationId = destinationId
        self.destinationName = destinationName
    
    def __str__(self):
        return "id:{},name:{}, sourceId:{}, sourceName:{}, destinationId:{}, destinationName:{}".format(self.id, self.name, self.sourceId ,self.sourceName , self.destinationId, self.destinationName)

dict_connections = CustomDict()
def loadConnections(pg):
    if type(pg) == int:
        return False
    else:
        connections = pg['processGroupFlow']['flow']['connections']
        for conection in connections:
            dict_connections.add(conection['component']['name'],
                                 Connection(
                                    conection['component']['id'],
                                    conection['component']['name'],
                                    conection['component']['source']['id'],
                                    conection['component']['source']['name'],
                                    conection['component']['destination']['id'],
                                    conection['component']['destination']['name']
                                 )
                                )
        return dict_connections

### Diccionario para tabla de procesadores

In [27]:
class TableProcessor:
    def __init__(self,name, tipo , status, flowFilesIn, flowFilesOut, taskCount):
        self.name = name
        self.tipo = tipo
        self.status = status
        self.flowFilesIn = flowFilesIn
        self.flowFilesOut = flowFilesOut
        self.taskCount = taskCount
    
    def __str__(self):
        return "name:{},tipo:{}, status:{}, flowFilesIn:{}, flowFilesOut:{}, taskCount:{}".format(self.name, self.tipo, self.status ,self.flowFilesIn , self.flowFilesOut, self.taskCount)


### diccionario para tabla de Connection

In [28]:
class TableConnection:
    def __init__(self, name, sourceName, destinationName, flowFilesQueued, queued, queuedCount, flowFilesIn, aceptados, flowFilesOut, procesados):
        ## aceptado = input
        ## procesadors = output
        self.name = name
        self.sourceName = sourceName
        self.destinationName = destinationName
        self.flowFilesQueued = flowFilesQueued
        self.queued = queued
        self.queuedCount = queuedCount
        self.flowFilesIn = flowFilesIn
        self.aceptado = aceptados
        self.flowFilesOut = flowFilesOut
        self.procesados =procesados
    
    def __str__(self):
        return "name:{}, sourceName:{}, destinationName:{}, flowFilesQueued:{}, queued:{}, queuedCount:{}, flowFilesIn:{}, aceptados:{}, flowFilesOut:{}, procesados:{}".format(self.name, self.sourceName, self.destinationName, self.flowFilesQueued, self.queued, self.queuedCount, self.flowFilesIn, self.aceptados, self.flowFilesOut, self.procesados)

### Diccionario para tabla de InputPort

In [ ]:
class TableInputPort:
    def __init__(self,name, tipo , status, flowFilesIn, flowFilesOut, taskCount):
        self.name = name
        self.tipo = tipo
        self.status = status
        self.flowFilesIn = flowFilesIn
        self.flowFilesOut = flowFilesOut
        self.taskCount = taskCount
    
    def __str__(self):
        return "name:{},tipo:{}, status:{}, flowFilesIn:{}, flowFilesOut:{}, taskCount:{}".format(self.name, self.tipo, self.status ,self.flowFilesIn , self.flowFilesOut, self.taskCount)


### Diccionario para tabla de Outputport:

In [ ]:
class TableOutputPort:
    def __init__(self,name, tipo , status, flowFilesIn, flowFilesOut, taskCount):
        self.name = name
        self.tipo = tipo
        self.status = status
        self.flowFilesIn = flowFilesIn
        self.flowFilesOut = flowFilesOut
        self.taskCount = taskCount
    
    def __str__(self):
        return "name:{},tipo:{}, status:{}, flowFilesIn:{}, flowFilesOut:{}, taskCount:{}".format(self.name, self.tipo, self.status ,self.flowFilesIn , self.flowFilesOut, self.taskCount)

### Diccionario  y lector de ProcessGroup anidado

In [29]:
class processGroupDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value

class processGroup:
    def __init__(self, name, uuid):
        self.name = name,
        self.uuid = uuid
    
    def __str__(self):
        return "name:{} ,uuid:{}".format(self.name, self.uuid)
    
dict_proccessgroup = processGroupDict()

def loadProcessgroups(pg):
    if type(pg) == int:
        return False
    else:
        proccessgroups = pg['processGroupFlow']['flow']['processGroups']
        for proccessgroup in proccessgroups:
            ##error add() takes 3 positional arguments but 4 were given puede ser por que dentro revision tiene dos valores
            dict_proccessgroup.add(proccessgroup['component']['name'], proccessgroup['component']['id'])                    
        return dict_proccessgroup

### Diccionarios de input port

In [44]:
class inputPortDict(dict):
    #__init___function
    def __init__(self):
        self = dict()
        
    #Function to add key:value
    def add(self, key, value):
        self[key] = value

def loadInputPort(pg):
    if type(pg) == int:
        return False
    else:
        inputPorts = pg['processGroupFlow']['flow']['inputPorts']
        inputPortDict.add(inputPorts['status']['aggregateSnapshot']['name'],  inputPorts['status']['aggregateSnapshot']['id'])   
        return inputPortDict



## Funciones proccessors

### GET Processors groups

In [31]:
#obtencion de datos del proccess groups
dict_tablep = TableDict()
def getProccessGroups(TokenNifi, id):
    flowProccessGroupsURI = URL + "/flow/process-groups/"
    #URL al process-group
    url =flowProccessGroupsURI+id
    
    #encabezado
    headers = {'Content-Type':'application/json', 'Authorization':'Bearer ' + TokenNifi}
    
    #almacena el request
    payload ={'uiOnly':True}
    response = requests.get(url, headers=headers, verify=False, params=payload)
    
    if response.status_code == 201 or response.status_code == 200: 
        response_json = response.json()
        
        return response_json
    else:
        return response.status_code

### GET Proccessor

In [32]:
# Traer los datos del proccesor
def GetProccessor(id):
    #agregar al url
    EnviarToken = "/processors/"+id
    #encabezado
    headers = {'Content-Type':'application/json', 'Authorization':'Bearer ' + TokenNifi}
    #almacena el requestzzz
    response = requests.get(URL+EnviarToken,headers=headers,verify=False)
    #Prueba
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        return response_json['revision']
    else:
        display(response.text)

### PLAY Proccessor

In [33]:
dict_tableP = CustomDict()
def PlayProccessor(uuid,revision):
    #agregar al url
    EnviarToken = "/processors/"+uuid+"/run-status"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    data={'revision': revision,"state": "RUNNING"}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=data)
    #Prueba
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        procesador = response_json["status"]
        
        dict_tableP.add(procesador["aggregateSnapshot"]['name']+"-Play",
                        TableProcessor(
                            procesador["aggregateSnapshot"]['name'],
                            procesador["aggregateSnapshot"]['type'],
                            procesador["aggregateSnapshot"]['runStatus'],
                            procesador["aggregateSnapshot"]['flowFilesIn'],
                            procesador["aggregateSnapshot"]['flowFilesOut'],
                            procesador["aggregateSnapshot"]['taskCount']
                        )
                    )
 
        return True
    else:
        return False

### STOP Proccessor

In [34]:
def StopProccessor(uuid,stoped):
    #agregar al url
    EnviarToken = "/processors/"+uuid+"/run-status"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    data={"revision": stoped,"state": "STOPPED"}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=data)
    #Prueba
    if response.status_code == 201 or response.status_code == 200: 
        response_json = response.json()
        procesador = response_json["status"]
        
        dict_tableP.add(procesador["aggregateSnapshot"]['name']+"-Stop",
                        TableProcessor(
                            procesador["aggregateSnapshot"]['name'],
                            procesador["aggregateSnapshot"]['type'],
                            procesador["aggregateSnapshot"]['runStatus'],
                            procesador["aggregateSnapshot"]['flowFilesIn'],
                            procesador["aggregateSnapshot"]['flowFilesOut'],
                            procesador["aggregateSnapshot"]['taskCount']
                        )
                    )
        return True
    else:
        return False

### CLEAR STATE Proccessor

In [35]:
def ClearProccessor(id):
    #aregar al url
    EnviarToken = "/processors/"+id+"/state/clear-requests"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    response = requests.post(URL+EnviarToken,headers=headers,verify=False)
    #Pueba
    if response.status_code == 201 or response.status_code == 200:            
        print("Datos del State limpiados")
        return True
    else:
        return False

## Connecctions

### Get connecction

In [36]:
dict_tableC = TableDict()
def GetConnecction(id):
    #agregar al url
    EnviarToken = "/connections/"+id
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    response = requests.get(URL+EnviarToken,headers=headers,verify=False)
    #Prueba
    if response.status_code == 201 or response.status_code == 200: 
        response_json = response.json()
        connections = response_json["status"]
        version = response_json["component"]['source']["running"]
        dict_tableC.add(connections["aggregateSnapshot"]['name']+str(version),
                        TableConnection(
                            connections["aggregateSnapshot"]['name'],
                            connections["aggregateSnapshot"]['sourceName'],
                            connections["aggregateSnapshot"]['destinationName'],
                            connections["aggregateSnapshot"]['flowFilesQueued'],
                            connections["aggregateSnapshot"]['queued'],
                            connections["aggregateSnapshot"]['queuedCount'],
                            connections["aggregateSnapshot"]['flowFilesIn'],
                            connections["aggregateSnapshot"]['input'],
                            connections["aggregateSnapshot"]['flowFilesOut'],
                            connections["aggregateSnapshot"]['output']
                        )
                    )
        return True
    else:
        return False

## funciones input

#### Get Input

In [47]:
TableInputPort = inputPortDict()
def getInputPort(id):
    EnviarToken = "/input-ports/"+id   
    headers = {'Content-Type':'application/json', 'Authorization':'Bearer ' + TokenNifi}
    #almacena el requestzzz
    response = requests.get(URL+EnviarToken,headers=headers,verify=False)
    #Prueba
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        return response_json['revision']
    else:
        display(response.text)    

#### Play Input

In [ ]:
def PlayInputPort(uuid,revision):
    #agregar al url
    EnviarToken = "/input-ports/"+uuid+"/run-status"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    data={'revision': revision,"state": "RUNNING"}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=data)
    #Prueba
    
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        inputPort = response_json["status"]
        
        TableInputPort.add(inputPort["aggregateSnapshot"]['name']+"-Play",
                        TableInputPort(
                            inputPort["aggregateSnapshot"]['name'],
                            inputPort["aggregateSnapshot"]['type'],
                            inputPort["aggregateSnapshot"]['runStatus'],
                            inputPort["aggregateSnapshot"]['flowFilesIn'],
                            inputPort["aggregateSnapshot"]['flowFilesOut'],
                            inputPort["aggregateSnapshot"]['taskCount']
                        )
                    ) 
        return True
    else:
        return False

#### Stop Input

In [ ]:
def StopInputPort(uuid,revision):
    #agregar al url
    EnviarToken = "/input-ports/"+uuid+"/run-status"
    #encabezado
    headers = {'Accept': '*/*','Content-Type':'application/json;charset=UTF-8','User-Agent': 'PostmanRuntime/7.29.0', 'Authorization':'Bearer '+TokenNifi}
    #almacena el request
    data={'revision': revision,"state": "STOPPED"}
    response = requests.put(URL+EnviarToken,headers=headers,verify=False, json=data)
    #Prueba
    
    if response.status_code == 201 or response.status_code == 200:
        response_json = response.json()
        inputPort = response_json["status"]
        
        TableInputPort.add(inputPort["aggregateSnapshot"]['name']+"-Play",
                        TableInputPort(
                            inputPort["aggregateSnapshot"]['name'],
                            inputPort["aggregateSnapshot"]['type'],
                            inputPort["aggregateSnapshot"]['runStatus'],
                            inputPort["aggregateSnapshot"]['flowFilesIn'],
                            inputPort["aggregateSnapshot"]['flowFilesOut'],
                            inputPort["aggregateSnapshot"]['taskCount']
                        )
                    ) 
        return True
    else:
        return False

## Generadores de tablas

### Tabla de procesadores

In [38]:
def informeTablaProcessor():
    diccionarios = dict(dict_tableP)
    x = PrettyTable(["Nombre", "Tipo", "Estatus", "flowFiles aceptados", "flowFiles transferido", "Tareas Ejecutadas"])

    for diccionario in diccionarios.keys():
        connection = dict_tableP.get(diccionario, "Processor no encontrado")
        conecetor = connection.__dict__
        x.add_row([conecetor["name"],conecetor["tipo"], conecetor["status"], conecetor["flowFilesIn"],conecetor["flowFilesOut"],conecetor["taskCount"]])
    print(x.get_string(title="Procesadores"))

### Tabla de conectores

In [39]:
def informeTablaconnector():
    diccionaries = dict(dict_tableC)
    y = PrettyTable(["Nombre", "Origen", "Destino","flowFiles en espera","En espera", "Cantidad en espera", "flowFiles aceptados", "Tamaño archivos aceptados","flowFiles transferido", "Tamaño archivos procesados"])
    for diccies in diccionaries.keys():
        connection = dict_tableC.get(diccies, "Processor no encontrado")
        conecetor = connection.__dict__
        y.add_row([conecetor["name"],conecetor["sourceName"],conecetor["destinationName"],conecetor["flowFilesQueued"],conecetor["queued"],conecetor["queuedCount"],conecetor["flowFilesIn"],conecetor["aceptado"],conecetor["flowFilesOut"],conecetor["procesados"]])
    print(y.get_string(title="Conectores"))


## Lectura del caso de prueba

In [45]:
def getIDfromLine(line):
    return line.split("'")[1]

def readFileInput():
    inputfile = open("pganidado.txt",'r')
    lines = inputfile.readlines()
    for line in lines:
        readEscenario(line)
        
def readEscenario(line):
    if "FEATURE" in line.upper():
        display("FEATURE "+getIDfromLine(line))
    elif "SCENARIO" in line.upper():
        display("SCENARIO "+getIDfromLine(line))
    elif "DATAFLOW" in line.upper():
               #UUID del proccessgroups ingresado en el archivo de txt
        uuid = getIDfromLine(line)
               ##obtenemos los datos del proccessgroup
        if getProccessGroups(TokenNifi, uuid) == False:            
            warnings.filterwarnings("ignore") 
            sys.exit('exiting...')
        else:
            loadProccessors(getProccessGroups(TokenNifi, uuid))
            loadConnections(getProccessGroups(TokenNifi, uuid))
            loadInputPort(getProccessGroups(TokenNifi, uuid))
        ### en get proccessGroups si sale 400 cerrar el ciclo con exit 
######################################################################################################################################################################################################        
    elif "PLAY" in line.upper():
        name = getIDfromLine(line)
        uuid = dict_proccessors.get(name, "Processor no encontrado")
        revision = GetProccessor(uuid)
        status_code = PlayProccessor(uuid,revision)
        if status_code == True:
            print("\n")
            print(">>>>> PLAY >>>>> " + name + " con UUID " + uuid)
            del revision
            del uuid
            del name
            time.sleep(5)
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
 ######################################################################################################################################################################################################         
    elif "STOP" in line.upper():
        name = getIDfromLine(line)
        uuid = dict_proccessors.get(name, "Processor no encontrado")        
        stoped = GetProccessor(uuid)
        status_code = StopProccessor(uuid,stoped)
        if status_code == True:
            print("\n")
            print(">>>>> STOP >>>>> " + name + " con UUID " + uuid)
            time.sleep(5)
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
########################################################################################################################################################################################################        
    elif "PROCESSGROUP" in line.upper():
        name = getIDfromLine(line)
        uuid = dict_proccessgroup.get(name, "ProcessGroup no encontrado")
        loadProccessors(getProccessGroups(TokenNifi, uuid))
        loadConnections(getProccessGroups(TokenNifi, uuid))
        loadConnections(getProccessGroups(TokenNifi, uuid))
        print(">>>>> ProcessGroup >>>>> " + name +"  "+ uuid)
        
        
########################################################################################################################################################################################################        
    elif "CHECK" in line.upper(): 
        
        name = getIDfromLine(line)        
        connection = dict_connections.get(name, "Conector no encontrado")
        if not type(connection) == str:
            conecetor = connection.__dict__
            flowfile = GetConnecction(conecetor['id'])
            if flowfile == True:
                time.sleep(3)
            else:
                try:
                    warnings.filterwarnings("ignore")            
                    sys.exit('exiting...')            
                except TypeError:
                    print("Los datos son incompatibles")
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
            
########################################################################################################################################################################################################        
    elif "CLEAR" in line.upper():
        name = getIDfromLine(line)
        uuid = dict_proccessors.get(name, "Processor no encontrado")        
        stoped = GetProccessor(uuid)
        status_code = ClearProccessor(uuid)
        if status_code == True:
            print("\n")
            print(">>>>> Clear >>>>> " + name + " con UUID " + uuid)
            time.sleep(5)
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
######################################################################################################################################################################################################        
    elif "PLAYINPUT" in line.upper():
        name = getIDfromLine(line)
        uuid = inputPortDict.get(name, "Input port no encontrado")
        revision = getInputPort(uuid)
        status_code = PlayInputPort(uuid,revision)
        if status_code == True:
            print("\n")
            print(">>>>> PLAY >>>>> " + name + " con UUID " + uuid)
            del revision
            del uuid
            del name
            time.sleep(5)
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
 ######################################################################################################################################################################################################         
    elif "STOPINPUT" in line.upper():
        name = getIDfromLine(line)
        uuid = inputPortDict.get(name, "Input poty no encontrado")   
        revision = getInputPort(uuid)
        status_code = StopProccessor(uuid,stoped)
        if status_code == True:
            print("\n")
            print(">>>>> STOP >>>>> " + name + " con UUID " + uuid)
            time.sleep(5)
        else:
            try:
                warnings.filterwarnings("ignore")            
                sys.exit('exiting...')            
            except TypeError:
                print("Los datos son incompatibles")
########################################################################################################################################################################################################
    elif "IMPRIMIR" in line.upper():
        
        Impresion = getIDfromLine(line)
        informeTablaProcessor()
        informeTablaconnector()
        
    
readFileInput()      

'FEATURE Pruebas de cluspac'

'SCENARIO Se espera ejecuciÃ³n exitosa'

[{'revision': {'clientId': 'bd67324e-1767-165c-d28e-8113e98f8357',
   'version': 9},
  'id': '1f83447f-0181-1000-ffff-ffffa7ed69c9',
  'uri': 'https://cdf01it-01adl.claro.amx:8443/nifi-api/input-ports/1f83447f-0181-1000-ffff-ffffa7ed69c9',
  'position': {'x': 1096.0, 'y': 1032.0},
  'permissions': {'canRead': True, 'canWrite': True},
  'bulletins': [],
  'component': {'id': '1f83447f-0181-1000-ffff-ffffa7ed69c9',
   'parentGroupId': '1f83071c-0181-1000-ffff-ffffe292ce2f',
   'position': {'x': 1096.0, 'y': 1032.0},
   'name': 'localtest',
   'state': 'STOPPED',
   'type': 'INPUT_PORT',
   'concurrentlySchedulableTaskCount': 1,
   'validationErrors': []},
  'status': {'id': '1f83447f-0181-1000-ffff-ffffa7ed69c9',
   'groupId': '1f83071c-0181-1000-ffff-ffffe292ce2f',
   'name': 'localtest',
   'transmitting': False,
   'runStatus': 'Stopped',
   'statsLastRefreshed': '10:08:59 ART',
   'aggregateSnapshot': {'id': '1f83447f-0181-1000-ffff-ffffa7ed69c9',
    'groupId': '1f83071c-0181-1000-f

>>>>> ProcessGroup >>>>> test  ProcessGroup no encontrado
